In [1]:
%%html
<style>
table {
    display: inline-block
}
</style>

# 20分钟内完成一个项目：随机短文发布系统

## 参考资源

资源名称 | 链接 | 说明
:-- | :-- | :--
fortunes | https://github.com/ruanyf/fortunes | code


## 1.项目目标
> 20分钟内完成一个**随机短文发布系统**，从开发到运行到测试全部完成。
<br> 1 分钟内完成部署+扩容

## 2.设计
> 使用软件 fortune 来生成随机短文：
<br> &emsp; &emsp; 执行 `fortune`命令，每运行一次都会输出随机的一段文字（唐诗、宋词、产品说明书等等）；
<br> 通过Linux脚本编写自动调用程序：
<br> &emsp; &emsp; 创建一个脚本每10秒调用一次执行，并将其输出存储在`index.html`中；
<br> 部署WEB服务器：
<br> &emsp; &emsp; 通过 Nginx Web服务器发布`index.html`短文内容。

## 3.代码
> 创建一个目录 `@fortune` 存储整个项目代码。

### 3.1.主页代码
> 创建一个网页：`@fortune/index.html`，代码如下：

```html
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
你好，我即将在这里发布短文......
</pre>
```

### 3.2.自动生成短文的脚本代码
> 创建一个脚本：`@fortune/fortuneloop.sh`，代码如下：

````shell
#!/bin/bash
trap "exit" SIGINT # 收到Ctrl c信息执行退出操作

while :
do
    echo '<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">' > /root/index.html
    echo '<pre>' >> /root/index.html
    /usr/games/fortune >> /root/index.html
    echo '</pre>' >> /root/index.html
    sleep 10
done
````

### 3.3.构建fortune镜像
> 创建一个镜像Dockerfile：`@fortune/Dockerfile`，代码如下：

````shell
FROM ubuntu
MAINTAINER htalker <flyjjbb@qq.com>
WORKDIR /root

# 使用国内安装源
RUN apt update && apt install --no-install-recommends --no-install-suggests -y ca-certificates \
  && cp -f /etc/apt/sources.list /etc/apt/sources.list.bak \
  && sed -i "s@http://.*archive.ubuntu.com@https://mirrors.tuna.tsinghua.edu.cn@g" /etc/apt/sources.list \
  && sed -i "s@http://.*security.ubuntu.com@https://mirrors.tuna.tsinghua.edu.cn@g" /etc/apt/sources.list \
  && apt update

# 设置中文运行环境
RUN apt install --no-install-recommends --no-install-suggests -y \
    locales \
    language-pack-zh-hans \
  && sed -i -e "s/# en_US.UTF-8 UTF-8/en_US.UTF-8 UTF-8/" /etc/locale.gen \
  && sed -i -e "s/# zh_CN.UTF-8 UTF-8/zh_CN.UTF-8 UTF-8/" /etc/locale.gen \
  && update-locale LANG=zh_CN.UTF-8 \
  && dpkg-reconfigure --frontend=noninteractive locales

# 安装应用
ADD fortuneloop.sh /usr/local/bin/fortuneloop.sh
RUN apt install --no-install-recommends --no-install-suggests -y \
     fortunes-zh \
  && chmod 755 /usr/local/bin/fortuneloop.sh

RUN apt clean && rm -rf /var/cache/apt/* /var/lib/apt/lists/*

# 中文环境变量
ENV LANG="zh_CN.UTF-8" LC_ALL="zh_CN.UTF-8"

ENTRYPOINT ["/usr/local/bin/fortuneloop.sh"]
# 覆盖父镜像的指令
CMD [""]
````

### 3.4.创建Docker容器编排配置文件
> 创建一个Docker容器编排脚本：`@fortune/compose.yml`，代码如下：

```yaml
version: "2"
services: 
  fortune:
    image: htalker/fortune:latest
    build:
      dockerfile: Dockerfile
    restart: "unless-stopped"
    tty: true
    user: "root:users"
    hostname: fortune
    network_mode: bridge
    volumes:
      - ./index.html:/root/index.html

  nginx:
    image: nginx:stable-alpine
    restart: "unless-stopped"
    tty: true
    user: "root:users"
    hostname: nginx
    ports:
      - "80:80"  #（Windows10以及较早版本是不能使用2000号以下的端口号）
    expose:
      - "80"
    network_mode: bridge
    volumes:
      # 主页
      - ./index.html:/usr/share/nginx/html/index.html
```

## 4.部署安装
> 注意：`dc` = `docker-compose`.

````shell
# 构建应用
cd @fortune
dc build fortune
# 启动应用
dc up -d
# 测试
curl localhost
# 上传应用
dc push fortune
# 卸载应用
dc down
````

## 5.系统扩容
> <img style="width: 60%;display: inline-block" src="img/Docker多实例.png">
<br> &emsp; 假设我们需要提供更多的WEB服务器，以应对更多的服务需求。比如我们需要部署5个WEB服务器。

### 1.Docker容器编排配置

```yaml
    ports:
      - "80:80"
# 由于一台主机上端口号不能有冲突，如果5个WEB服务器需要5个端口号，我们使用如下的指令提供 80-84之间连续端口号。
    ports:
      - "80-84:80" # 只需要修改本行
```
> 怪异问题：以上的编排配置随机性的报错，需要多重试几次。

### 2.指令

````shell
cd @fortune
# 卸载之前的实例
dc down

# 测试指令，看看端口号是否有冲突 8?
netstat -ano | find ":8"

# 构建应用(数据生成1个实例，Web服务5个实例)
dc up -d --scale nginx=5 --scale fortune=1
[+] Running 6/6
✔ Container fortune-nginx-5    Started 1.6s
✔ Container fortune-nginx-2    Started 1.4s
✔ Container fortune-fortune-1  Started 0.5s
✔ Container fortune-nginx-1    Started 0.6s
✔ Container fortune-nginx-4    Started 1.9s
✔ Container fortune-nginx-3    Started 1.1s

# 查看容器实例
dc ps
NAME                IMAGE                    COMMAND                   SERVICE   CREATED          STATUS          PORTS
fortune-fortune-1   htalker/fortune:latest   "/usr/local/bin/fort…"   fortune   24 minutes ago   Up 24 minutes
fortune-nginx-1     nginx:stable-alpine      "/docker-entrypoint.…"   nginx     24 minutes ago   Up 24 minutes   0.0.0.0:84->80/tcp
fortune-nginx-2     nginx:stable-alpine      "/docker-entrypoint.…"   nginx     24 minutes ago   Up 24 minutes   0.0.0.0:80->80/tcp
fortune-nginx-3     nginx:stable-alpine      "/docker-entrypoint.…"   nginx     24 minutes ago   Up 24 minutes   0.0.0.0:81->80/tcp
fortune-nginx-4     nginx:stable-alpine      "/docker-entrypoint.…"   nginx     24 minutes ago   Up 24 minutes   0.0.0.0:83->80/tcp
fortune-nginx-5     nginx:stable-alpine      "/docker-entrypoint.…"   nginx     24 minutes ago   Up 24 minutes   0.0.0.0:82->80/tcp

# 端口号
netstat -ano | find ":8"
  TCP    0.0.0.0:80             0.0.0.0:0              LISTENING       14028
  TCP    0.0.0.0:81             0.0.0.0:0              LISTENING       14028
  TCP    0.0.0.0:82             0.0.0.0:0              LISTENING       14028
  TCP    0.0.0.0:83             0.0.0.0:0              LISTENING       14028
  TCP    0.0.0.0:84             0.0.0.0:0              LISTENING       14028

# 测试
curl localhost
curl localhost:81
curl localhost:82
curl localhost:83
curl localhost:84
# 卸载应用
dc down
````